In [1]:
# all imports and constant definition

import pandas as pd
import numpy as np

import re
import glob
import os
import os.path
import sys

DAILY_DATA_FOLDER = 'E:/analytics/stock/hist-D-20160501-0707'
DAILY_SAMPLE_PATH = 'E:/analytics/stock/hist/002415.csv'

WEEKLY_DATA_FOLDER = 'E:/analytics/stock/hist-W-20160401-0709'
WEEKLY_SAMPLE_PATH = 'E:/analytics/stock/hist-W-2016-0627/002643.csv'

DATE_FOR_PAUSE_CHECK = '2016-07-07'

pd.set_option('max_rows', 20)

In [3]:
df = pd.read_csv(DAILY_SAMPLE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-04-22,31.17,2.37,30.686,30.857,30.809
2016-04-21,30.45,0.79,30.732,30.806,30.803
2016-04-20,30.21,-1.60,30.880,30.831,30.875
2016-04-19,30.70,-0.65,31.064,30.946,30.953
2016-04-18,30.90,-1.59,31.004,31.016,31.038
2016-04-15,31.40,0.67,31.028,30.976,31.037
2016-04-14,31.19,0.19,30.880,30.918,30.967
2016-04-13,31.13,2.40,30.782,30.909,30.868
2016-04-12,30.40,-2.00,30.828,30.781,30.792


In [5]:
r = df.resample('W')
# daily average price change percent on weekly basis
r['p_change'].mean().dropna().sort_values()

date
2015-07-12   -5.413333
2015-06-21   -3.862000
2013-12-08   -2.440000
2015-03-08   -2.382500
2014-03-23   -2.336000
                ...   
2015-04-05    2.442000
2013-06-16    2.645000
2014-12-07    2.842000
2015-01-11    3.418000
2015-05-24    6.086000
Name: p_change, dtype: float64

In [6]:
df = pd.read_csv(WEEKLY_SAMPLE_PATH, index_col=0, parse_dates=True, \
                 usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                 error_bad_lines=False)
df

,close,p_change,ma5,ma10,ma20
date,,,,,
2016-06-27,58.10,3.11,52.744,50.356,42.310
2016-06-24,56.35,7.33,50.660,48.755,41.205
2016-06-17,52.50,6.56,49.130,47.041,40.087
2016-06-08,49.27,3.73,48.250,45.011,39.300
2016-06-03,47.50,-0.38,47.926,43.202,38.561
2016-05-27,47.68,-2.09,47.968,41.792,38.409
2016-05-20,48.70,1.25,46.850,40.089,38.051
2016-05-13,48.10,0.94,44.952,38.317,37.742
2016-05-06,47.65,-0.13,41.772,36.678,37.301


In [14]:
df.index.get_loc(df[df['p_change']<0].index[0])
df.index[0].dayofweek

0

In [7]:
r_df = df.sort_index()
r_df.rolling(window=5)['close'].mean()

date
2016-01-08       NaN
2016-01-15       NaN
2016-01-22       NaN
2016-01-29       NaN
2016-02-05    37.140
2016-02-19    35.772
2016-02-26    35.592
2016-03-04    34.584
2016-03-11    33.980
2016-03-18    32.910
               ...  
2016-04-29    38.478
2016-05-06    41.772
2016-05-13    44.952
2016-05-20    46.850
2016-05-27    47.968
2016-06-03    47.926
2016-06-08    48.250
2016-06-17    49.130
2016-06-24    50.660
2016-06-27    52.744
Name: close, dtype: float64

In [6]:
''' detect increase trend by use of MA, applicable for both daily (kind: 'd') and weekly data (kind: 'w')
'''
def find_increase_trend(df, trend_threshold=3):
    
    if len(df) == 0:
        return None
    
    s = df['ma5']
    # the position from which (exclusive) MA starts to rise until latest
    ma_change_point = -1
    for i in range(0, len(s)-1):
        if s[i] < s[i+1]:
            ma_change_point = i 
            break
    else:
        ma_change_point = len(s) - 1
    
    # the position from which (exclusive) closing price starts to rise until latest
    # 0 means it has been falling in recent period, may rise 'tomorrow'
    if not (df['p_change']<0).any():
        starting_rise_point = len(df)-1
    else:
        starting_rise_point = df.index.get_loc(df[df['p_change']<0].index[0])
    
    trend_start = starting_rise_point if starting_rise_point > ma_change_point else ma_change_point
    # a trend is established at least lasting for 3 occurrences
    if trend_start < trend_threshold:
        return None
    else:
        startdate = df.index[trend_start]
        # use close price of two ends to calc total increase percent
        total_increase = round(df.iat[0, 0] / df.iat[trend_start, 0] - 1, 4) * 100
        # the observation at change point doesn't count as increase, but only later ones
        trend_length = trend_start
        # mean week-over-week increase percent over this period
        mean_increase = round(df[:trend_start]['p_change'].mean(), 2)
        first_above_mean_position = df.index.get_loc(df[df['p_change'] >= mean_increase].index[0])
        # num of recent consecutive obserations whose WoW increase is lower than mean
        recent_below_mean_count = first_above_mean_position
        std = round(df[:trend_start]['p_change'].std(), 2)
        return startdate, total_increase, trend_length, mean_increase, std, recent_below_mean_count

In [11]:
find_increase_trend(WEEKLY_DATA_FOLDER + '/002800.csv')

(Timestamp('2016-06-03 00:00:00'),
 527.96000000000004,
 5,
 45.84,
 22.04,
 True,
 1)

In [5]:
pd.Timestamp('20161001') > pd.Timestamp('20000101')

True

In [7]:
def analyze_trend(folder, kind='w', trend_threshold=3, increase_threshold=10, max_recent_slowdown=1):
    reg = re.compile(r'(\d{6}).csv')
    stocks = {t[1].group(1):t[0] for t in ((x, reg.search(x)) for x in glob.glob(folder + '/*.csv')) if t[1]}
    # [os.path.isfile(x) for x in list(stocks.values())[:5]]
    resultmap = {}
    latest = pd.Timestamp('20000101')
    for code in stocks:
        try: 
#             print('processing ', code)
            df = pd.read_csv(stocks[code], index_col=0, parse_dates=True, \
                             usecols=['date', 'close', 'p_change', 'ma5', 'ma10', 'ma20'], \
                             error_bad_lines=False)
            if len(df) == 0:
                continue
            
            # delete first entry if it doesn't stand for weekly data (whose timestamp should be Fri)
            # usually daily data for date at retrieval is also collected
            if kind == 'w' and df.index[0].dayofweek != 4:
                df = df[1:]
                
            # latest date available in input stock data,
            # absence of it indicates the stock's tranding is paused at that time
            if df.index[0] > latest:
                latest = df.index[0]
            elif df.index[0] < latest:
                continue
                
            res = find_increase_trend(df, trend_threshold)
            ''' take as valid entry when following conditions are met:
                    1. increasing trend lasts longer than 3 observations 
                    2. actual increase percent over the period is above 10% (MA trails behind actual varation)
                    3. increasing trend didn't considerably slowdown lately
            '''
            if res and res[1] > increase_threshold and res[-1] <= max_recent_slowdown:
                resultmap[code] = res
        except Exception as ex:
            print('error occurred in processing %s: %s' % (code, ex))
    df = pd.DataFrame.from_dict(resultmap, orient='index')
    df.columns = ['startdate', 'increase', 'length', 'mean', 'std', 'RSL']
    print(df.head())
    return df

In [8]:
TREND_ANALYSIS_OUTPUT_PATH = 'E:/analytics/stock/analysis_weekly_0704.csv'
df = analyze_trend(DAILY_DATA_FOLDER)
print('done analyzing trend analysis, found %d records' % len(df))

error occurred in processing 002272: index 0 is out of bounds for axis 0 with size 0
error occurred in processing 002805: index 0 is out of bounds for axis 0 with size 0
error occurred in processing 600556: index 0 is out of bounds for axis 0 with size 0
error occurred in processing 000968: index 0 is out of bounds for axis 0 with size 0
        startdate  increase  length  mean   std  RSL
000760 2016-06-29     13.15       5  2.59  4.90    0
000536 2016-06-17     12.76      13  0.97  3.04    1
002352 2016-06-28     13.32       6  2.21  5.03    0
000603 2016-06-24     44.09       8  4.75  4.52    1
600435 2016-06-30     21.26       4  5.02  4.50    1
done analyzing trend analysis, found 315 records


In [ ]:
df.to_csv(TREND_ANALYSIS_OUTPUT_PATH)

In [9]:
pd.set_option('max_rows', 20)

BASIC_DATA_PATH = r'E:\analytics\stock\basics\basic.csv'
basic_df = pd.read_csv(BASIC_DATA_PATH, index_col=False, dtype={'code':np.str}, \
                       usecols=['code', 'pe', 'pb', 'outstanding', 'totals', 'esp', 'timeToMarket'], \
                       error_bad_lines=False)
basic_df.set_index('code', inplace=True)
basic_df['timeToMarket'] = pd.to_datetime(basic_df['timeToMarket'], errors='coerce', format='%Y%m%d')
basic_df = basic_df[basic_df['timeToMarket'].notnull()]
basic_df

,pe,outstanding,totals,esp,pb,timeToMarket
code,,,,,,
002805,30.20,2422.89,9691.37,0.069,1.47,2016-07-07
600467,217.03,146099.44,146099.44,0.006,2.38,2004-04-05
000795,402.10,65279.01,113368.40,0.006,7.45,1997-08-08
000758,70.18,180298.09,196937.84,0.033,3.85,1997-04-16
000890,554.74,37956.57,37964.16,0.005,4.28,1999-01-19
000919,39.62,50316.89,50400.00,0.095,3.04,1999-11-18
600848,1264.57,29214.14,89517.21,0.004,5.35,1994-03-24
000551,83.11,40008.03,40008.03,0.030,3.02,1994-01-06
002738,0.00,8784.29,18691.50,-0.007,8.70,2014-12-30


In [10]:
REPORT_DATA_FOLDER = r'E:/analytics/stock/report'

reg = re.compile(r'(\d{4}-\d).csv')
reports = {t[1].group(1):t[0] for t in ((x, reg.search(x)) for x in glob.glob(REPORT_DATA_FOLDER + '/*.csv')) if t[1]}
report_dfs = []
report_terms = []
for term in reports:
    rdf = pd.read_csv(reports[term], index_col=False, dtype={'code':np.str}, \
                           usecols=['code', 'roe', 'profits_yoy'], \
                           error_bad_lines=False)
    rdf.set_index('code', inplace=True)
    d = rdf.index.duplicated()
#     print('duplicates in %s: %d (%s)' % (term, len(d[d==True]), rdf.index[d==True][:5]))
    rdf.drop_duplicates(inplace=True)
    report_dfs.append(rdf)
    report_terms.append(term)
# print(report_dfs)
all_report_df = pd.concat(report_dfs, keys=report_terms, axis=1, join='outer')
all_report_df

2016-1             2015-2             2015-4             2015-3  \
          roe profits_yoy    roe profits_yoy    roe profits_yoy    roe   
000001   3.70        8.12    NaN         NaN  14.94       10.42  12.39   
000002   0.83       28.14   5.41        0.77  19.14       15.08   7.69   
000004    NaN         NaN   1.50     -125.76   1.54      -67.19   2.92   
000005  -0.59      -15.65  -3.52       -8.05  -8.07     -227.98  -4.56   
000006   1.11      -75.79    NaN         NaN   9.77      -17.97   6.56   
000007   2.05     -257.50  -2.12       89.72   3.87     -146.35  -3.35   
000008   0.24      -79.46   2.93      710.43   7.60     2290.28    NaN   
000009    NaN         NaN   3.38      -48.53  19.70      160.12  17.55   
000010  -1.51     1407.81   0.29     -125.35   4.05     -144.86   0.50   
000011    NaN         NaN   1.52      -72.19   7.59      -62.44    NaN   
...       ...         ...    ...         ...    ...         ...    ...   
603968   1.74      -47.32  10.41       23.49  12.34        4.24  11.43   
603969    NaN         NaN   5.57       -6.31   9.51       -2.22    NaN   
603979   1.23       -4.54   6.84       -9.71   7.70      -27.79   7.94   
603988   1.81      -22.82   3.82      -22.99   6.89      -21.29   5.74   
603989    NaN         NaN   9.36      -13.29  16.52       25.14  12.12   
603993   0.81      -53.67   3.15      -53.92   4.77      -58.28    NaN   
603996   0.19      -75.55   6.19         NaN  13.18        5.15    NaN   
603997   4.36       13.74   8.32      -11.73    NaN         NaN  11.55   
603998    NaN         NaN    NaN         NaN  10.75       11.96   8.62   
603999   0.74      -20.19   4.20         NaN   9.86      -28.07    NaN   

                    
       profits_yoy  
000001       13.04  
000002        6.14  
000004     -142.69  
000005       -6.61  
000006       15.54  
000007       74.22  
000008         NaN  
000009      160.90  
000010     -120.03  
000011         NaN  
...            ...  
603968        1.05  
603969         NaN  
603979      -13.87  
603988      -16.53  
603989       11.20  
603993         NaN  
603996         NaN  
603997      -12.44  
603998       15.08  
603999         NaN  

[2864 rows x 8 columns]

In [11]:
CONSOLIDATED_DATA_PATH = r'E:/analytics/stock/consolidated-D-0710.csv'
consolidated = df.join(basic_df).join(all_report_df)
consolidated.to_csv(CONSOLIDATED_DATA_PATH)